Experimento 9 - Linear Interpolation Imputer, 3sigma, min max normalization, knn classifier

In [15]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import seaborn.objects as so
from ucimlrepo import fetch_ucirepo

In [16]:
beans = fetch_ucirepo(id=602)
df = beans.data.features
targets = beans.data.targets

In [17]:
cols = ['Area', 'Perimeter', 'MajorAxisLength', 'MinorAxisLength', 'AspectRatio', 'Eccentricity', 'ConvexArea', 'EquivDiameter', 'Extent', 'Solidity', 'Roundness', 'Compactness', 'ShapeFactor1', 'ShapeFactor2', 'ShapeFactor3', 'ShapeFactor4']

In [18]:
import random

#Introducing Missing values (5%)

for index, i in enumerate(df):
  for jndex, j in enumerate(df[i]):
    if random.randint(0,100) < 5:
      df.loc[jndex,i] = np.NaN

<ipython-input-18-fbb7389abead>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[i][jndex] = np.NaN


In [19]:
df.isna().sum()

Area               662
Perimeter          625
MajorAxisLength    706
MinorAxisLength    673
AspectRatio        666
Eccentricity       725
ConvexArea         696
EquivDiameter      716
Extent             656
Solidity           706
Roundness          638
Compactness        687
ShapeFactor1       667
ShapeFactor2       614
ShapeFactor3       675
ShapeFactor4       652
dtype: int64

In [20]:
df

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRatio,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,Roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4
0,28395.0,610.291,208.178117,173.888747,1.197191,0.549812,28715.0,190.141097,0.763923,0.988856,0.958027,0.913358,0.007332,0.003147,0.834222,0.998724
1,28734.0,638.018,200.524796,182.734419,1.097356,0.411785,29172.0,191.272751,0.783968,0.984986,0.887034,0.953861,0.006979,0.003564,0.909851,0.998430
2,NaN,624.110,212.826130,175.931143,1.209713,0.562727,29690.0,193.410904,0.778113,0.989559,0.947849,0.908774,0.007244,0.003048,0.825871,0.999066
3,30008.0,645.884,210.557999,182.516516,1.153638,0.498616,30724.0,195.467062,0.782681,0.976696,0.903936,0.928329,0.007017,0.003215,0.861794,0.994199
4,30140.0,620.134,201.847882,190.279279,1.060798,0.333680,30417.0,195.896503,0.773098,0.990893,0.984877,NaN,0.006697,0.003665,0.941900,0.999166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13606,42097.0,759.696,288.721612,185.944705,1.552728,0.765002,NaN,231.515799,0.714574,0.990331,0.916603,0.801865,NaN,0.001749,0.642988,0.998385
13607,42101.0,757.499,281.576392,190.713136,1.476439,0.735702,NaN,231.526798,0.799943,0.990752,0.922015,0.822252,NaN,0.001886,0.676099,0.998219
13608,NaN,759.321,281.539928,191.187979,1.472582,0.734065,NaN,231.631261,0.729932,0.989899,0.918424,0.822730,0.006681,0.001888,0.676884,0.996767
13609,42147.0,763.779,283.382636,190.275731,1.489326,0.741055,42667.0,231.653247,0.705389,0.987813,0.907906,NaN,0.006724,0.001852,0.668237,0.995222


In [1]:
#Imputing Missing Values with Linear Interpolation
df_imputed = df.interpolate(method="nearest", order=3, limit=None,
                            limit_direction='both').ffill().bfill()
df_imputed

NameError: ignored

In [22]:
#adding the labels
df_imputed['Class'] = targets
df_pre_missing_values = df.copy()
print(df_imputed.groupby('Class').agg(['count']))
df_imputed['Class'] = df_imputed['Class'].transform(lambda x: 0 if x == 'BARBUNYA' else (1 if x == 'BOMBAY' else (2 if x == 'CALI' else (3 if x == 'DERMASON' else (4 if x == 'HOROZ' else (5 if x == 'SEKER' else 6))))))

          Area Perimeter MajorAxisLength MinorAxisLength AspectRatio  \
         count     count           count           count       count   
Class                                                                  
BARBUNYA  1322      1322            1322            1322        1322   
BOMBAY     522       522             522             522         522   
CALI      1630      1630            1630            1630        1630   
DERMASON  3546      3546            3546            3546        3546   
HOROZ     1928      1928            1928            1928        1928   
SEKER     2027      2027            2027            2027        2027   
SIRA      2636      2636            2636            2636        2636   

         Eccentricity ConvexArea EquivDiameter Extent Solidity Roundness  \
                count      count         count  count    count     count   
Class                                                                      
BARBUNYA         1322       1322          1322   13

In [23]:
#Outlier Removal with 3sigma
from scipy import stats


print(f'Pre Outlier Shape: {df_imputed.shape}')
df_no_outliers = df_imputed.copy()

print(df_no_outliers['Class'].unique())
for i in df_no_outliers['Class'].unique():
    class_unique = df_no_outliers[df_no_outliers['Class'] == i]
    for feature in class_unique:
      upper = class_unique[feature].mean() + (3 * class_unique[feature].std())
      lower = class_unique[feature].mean() - (3 * class_unique[feature].std())
      excluded = pd.Series(class_unique[class_unique[feature] < lower].index)
      #print(excluded.values)
      df_no_outliers.drop(excluded.values, inplace = True, errors='ignore')

print(df_no_outliers.groupby('Class').count())
print(f'Pos Outlier Shape: {df_no_outliers.shape}')


Pre Outlier Shape: (13611, 17)
[5 0 1 2 4 6 3]
       Area  Perimeter  MajorAxisLength  MinorAxisLength  AspectRatio  \
Class                                                                   
0      1260       1260             1260             1260         1260   
1       500        500              500              500          500   
2      1579       1579             1579             1579         1579   
3      3404       3404             3404             3404         3404   
4      1812       1812             1812             1812         1812   
5      1915       1915             1915             1915         1915   
6      2538       2538             2538             2538         2538   

       Eccentricity  ConvexArea  EquivDiameter  Extent  Solidity  Roundness  \
Class                                                                         
0              1260        1260           1260    1260      1260       1260   
1               500         500            500     500    

In [24]:
df_no_outliers['Class'] = df_no_outliers['Class'].transform(lambda x: 'BARBUNYA' if x == 0 else ('BOMBAY' if x == 1 else ('CALI' if x == 2 else ('DERMASON' if x == 3 else ('HOROZ' if x == 4 else ('SEKER' if x == 5 else 'SIRA'))))))
df_no_outliers = df_no_outliers.reset_index()
print(df_no_outliers.groupby('Class').agg(['count']))

         index  Area Perimeter MajorAxisLength MinorAxisLength AspectRatio  \
         count count     count           count           count       count   
Class                                                                        
BARBUNYA  1260  1260      1260            1260            1260        1260   
BOMBAY     500   500       500             500             500         500   
CALI      1579  1579      1579            1579            1579        1579   
DERMASON  3404  3404      3404            3404            3404        3404   
HOROZ     1812  1812      1812            1812            1812        1812   
SEKER     1915  1915      1915            1915            1915        1915   
SIRA      2538  2538      2538            2538            2538        2538   

         Eccentricity ConvexArea EquivDiameter Extent Solidity Roundness  \
                count      count         count  count    count     count   
Class                                                              

In [25]:
df_no_outliers = df_no_outliers.drop(['index'], axis='columns')
df_no_outliers

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRatio,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,Roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,28395.0,610.291,208.178117,173.888747,1.197191,0.549812,28715.00,190.141097,0.763923,0.988856,0.958027,0.913358,0.007332,0.003147,0.834222,0.998724,SEKER
1,28734.0,638.018,200.524796,182.734419,1.097356,0.411785,29172.00,191.272751,0.783968,0.984986,0.887034,0.953861,0.006979,0.003564,0.909851,0.998430,SEKER
2,29371.0,624.110,212.826130,175.931143,1.209713,0.562727,29690.00,193.410904,0.778113,0.989559,0.947849,0.908774,0.007244,0.003048,0.825871,0.999066,SEKER
3,30279.0,634.927,212.560556,181.510182,1.171067,0.520401,30600.00,196.347702,0.775688,0.989510,0.943852,0.923726,0.007020,0.003153,0.853270,0.999236,SEKER
4,30477.0,670.033,211.050155,184.039050,1.146768,0.489478,30970.00,196.988633,0.762402,0.984081,0.853080,0.924603,0.006925,0.003242,0.871186,0.999049,SEKER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13003,42097.0,759.696,288.721612,185.944705,1.552728,0.765002,42510.25,231.515799,0.714574,0.990331,0.916603,0.801865,0.006612,0.001749,0.642988,0.998385,DERMASON
13004,42101.0,757.499,281.576392,190.713136,1.476439,0.735702,42562.50,231.526798,0.799943,0.990752,0.922015,0.822252,0.006646,0.001886,0.676099,0.998219,DERMASON
13005,42124.0,759.321,281.539928,191.187979,1.472582,0.734065,42614.75,231.631261,0.729932,0.989899,0.918424,0.822730,0.006681,0.001888,0.676884,0.996767,DERMASON
13006,42147.0,763.779,283.382636,190.275731,1.489326,0.741055,42667.00,231.653247,0.705389,0.987813,0.907906,0.803863,0.006724,0.001852,0.668237,0.995222,DERMASON


In [26]:
#Normalization with min max
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit_transform(df_no_outliers[cols])
df_scaled = pd.DataFrame(scaler.transform(df_no_outliers[cols]), columns = cols)

df_scaled['Class'] = df_no_outliers['Class']
df_scaled

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRatio,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,Roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,0.034338,0.060789,0.040030,0.137895,0.087594,0.325651,0.033600,0.071030,0.671024,0.779503,0.902711,0.848057,0.635649,0.861196,0.822198,0.961646,SEKER
1,0.035806,0.080647,0.026167,0.165422,0.013725,0.068251,0.035533,0.073829,0.735504,0.632906,0.691220,0.973978,0.585538,1.000000,0.968892,0.950484,SEKER
2,0.038564,0.070686,0.048449,0.144251,0.096859,0.349735,0.037724,0.079116,0.716671,0.806121,0.872391,0.833807,0.623209,0.828005,0.805998,0.974660,SEKER
3,0.042495,0.078433,0.047968,0.161612,0.068264,0.270803,0.041574,0.086379,0.708871,0.804267,0.860482,0.880291,0.591419,0.863027,0.859144,0.981111,SEKER
4,0.043352,0.103576,0.045232,0.169481,0.050285,0.213136,0.043139,0.087964,0.666131,0.598657,0.590072,0.883018,0.577903,0.892778,0.893896,0.973999,SEKER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13003,0.093662,0.167792,0.185924,0.175412,0.350663,0.726947,0.091956,0.173345,0.512286,0.835379,0.779308,0.501434,0.533422,0.395047,0.451264,0.948770,DERMASON
13004,0.093680,0.166218,0.172982,0.190250,0.294215,0.672307,0.092177,0.173373,0.786890,0.851303,0.795431,0.564816,0.538359,0.440635,0.515489,0.942435,DERMASON
13005,0.093779,0.167523,0.172916,0.191728,0.291361,0.669254,0.092398,0.173631,0.561689,0.818998,0.784733,0.566301,0.543296,0.441448,0.517013,0.887246,DERMASON
13006,0.093879,0.170716,0.176254,0.188889,0.303751,0.682289,0.092619,0.173685,0.482741,0.739982,0.753401,0.507647,0.549325,0.429363,0.500239,0.828503,DERMASON


In [27]:
#Classification with Knn-Classification

from sklearn.neighbors import KNeighborsClassifier as knnc
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn import preprocessing

lb = preprocessing.LabelBinarizer()

classifier = knnc(n_neighbors=10)
x_train = df_scaled.iloc[:, 0:16]
y_train = df_scaled.iloc[:, 16]
y_train = np.array([number[0] for number in lb.fit_transform(y_train)])
scoring = {'acc' : 'accuracy',
           'prec' : 'precision_macro',
           'recall' : 'recall_macro',
           'f1' : 'f1_macro'}

y_pred = cross_validate(classifier, x_train, y_train, cv=10, scoring=scoring, return_train_score=True)


print('Experimento 9')
print('Acurácia Média: ' + '%.2f' % (np.mean(y_pred['test_acc'])*100) + '%')
print('Precisão Média: ' + '%.2f' % (np.mean(y_pred['test_prec'])*100) + '%')
print('Revocação Média: ' + '%.2f' % (np.mean(y_pred['test_recall'])*100) + '%')
print('F1-Score Médio: ' + '%.2f' % (np.mean(y_pred['test_f1'])*100) + '%')

Experimento 9
Acurácia Média: 96.98%
Precisão Média: 94.66%
Revocação Média: 93.01%
F1-Score Médio: 92.85%
